# Continuing simulations
This notebook will show you:
- how to continue an existing ANN assisted TPS simulation from storage

Note: This notebook depends on files created in the first notebook `1_Toy_pytorch_simple_setup.ipynb`. Please do it first if you have not already.

In [1]:
%matplotlib inline

In [2]:
import os
import arcd
import torch
import numpy as np
import matplotlib.pyplot as plt
import openpathsampling as paths

Using TensorFlow backend.


In [3]:
# change to the working directory of choice
# (should be the same as in the first notebook)
wdir = '/homeloc/scratch/hejung/arcd_scratch/toys/'
#wdir = None
if wdir is not None:
    os.chdir(wdir)

### The easy way
By continuing an `openpathsampling` TPS simulation that has been set up with arcd you will autmatically load the `RCModel` saved automagically after the simulation. This will furhtermore repopulate the attached `TrainSet` with the shooting results from the `openpathsampling.Storage`.
To do this, simply continue a TPS simulation the OPS way.

In [4]:
# open old storage for appending
storage = paths.Storage('22dim_toy_pytorch.nc', 'a')

In [5]:
# get the PathSampling from storage and set its state to the last MCStep in storage
sampler = storage.pathsimulators[0]
sampler.restart_at_step(storage.steps[-1])

Restoring RCModelSelector without model.If used together with arcd.TrainingHook you can ignore this warning, otherwise please take care of resetting the model yourself.


In [6]:
# create the empty training hook,
# the `arcd.TrainHook` is the only object which will not be saved,
# but it also has no internal state except for the save_interval parameter
trainhook = arcd.ops.TrainingHook(None, None, 100)
sampler.attach_hook(trainhook)

In [7]:
# Note that this did not yet load the RCModel nor repopulate the TrainSet
print('trainhook.model: ', trainhook.model)
print('trainhook.trainset: ', trainhook.trainset)

trainhook.model:  None
trainhook.trainset:  None


In [8]:
# if we now do one step of TPS it will repopulate the TrainSet and load the model
# the reason is that we use the `before_simulation` hook of `openpathsampling.Pathsimulator` to load the state
sampler.run(1)
# you can ignore the warning about the MCstep without a Shooting snapshot, the first OPS step has no shooting snap

Working on Monte Carlo cycle number 501
Starting simulation...
Working on first step
DONE! Completed 501 Monte Carlo cycles.


In [9]:
# Now trainset and model are populated and you can do everything you learned in the last notebook
print('trainhook.model: ', trainhook.model)
print('trainhook.trainset: ', trainhook.trainset)

trainhook.model:  <arcd.pytorch.rcmodel.EEPytorchRCModel object at 0x7f2074149240>
trainhook.trainset:  <arcd.base.trainset.TrainSet object at 0x7f1fe9487ac8>


In [10]:
storage.sync_all()
storage.close()